In [1]:
import os
import numpy as np
import pandas as pd
import gensim.models
from gensim import utils
from graph import *
import networkx as nx
import random

In [47]:
class Corpus:
    
    def __init__(self, pt, groundTruth, testProp, p, q):
        self.path = pt
        print('building train/test sets')
        self.build(pt, testProp,groundTruth)
        self.p = p
        self.q = q
        print('building graph')
        self.generateCorpus()
        
        pass
    
    def build(self, pt, testProp, groundTruth):
        #note: temp1 is the training/test set; temp2 is the graph
        df = pd.read_csv(pt).astype(str) 
        temp1 = pd.read_csv(groundTruth)
        print(list(temp1.columns))
        temp1 = self.generateNegativeSamples(temp1)
        self.trainTestSplit(temp1, testProp)
        self.buildGraph(df)
    
    def generateNegativeSamples(self,df):
        df['label'] = 1
        col1 = df.columns[0]
        col2 = df.columns[1]
        allItems = set(df[col1])
        negSamples = []
        for i, row in df.iterrows():
            source = row[col1]
            target = row[col2]
            negSamples.append((source, random.sample(allItems.difference(set([target])), 1)[0]))
        t = pd.DataFrame(negSamples, columns = [col1,col2])
        t['label'] = 0
        return pd.concat([t, df])
    
    
    def trainTestSplit(self, df, testProp):
        df = df.sample(len(df))
        nodes = set(df[df.columns[1]].values)
        nodes.update(set(df[df.columns[0]].values))
        self.nodes = [str(x) for x in list(nodes)]
        cutoff = int(len(df) * testProp)
        test = df.iloc[0:643]
        train = df.iloc[643:]
        self.train = train
        self.test = test
    
    def buildGraph(self, df):
        G=nx.Graph()
        
        temp = set([str(x) for x in df['source']])
        temp.update(set([str(x) for x in df['target']]))
        edges = tuple(zip([str(x) for x in list(df['source'])], [str(x) for x in list(df['target'])]))
        G.add_nodes_from(temp)
        G.add_edges_from(edges, weight = 1)
        self.G = G
    
    def generateCorpus(self):
        self.corpus = Graph(self.G, self.nodes, is_directed= False, p = self.p, q = self.q)
        self.corpus.preprocess_transition_probs()

In [48]:
class Model:
    
    def __init__(self, corpus, embedder = None, model = None):
        self.corpus = corpus
        self.embedder = embedder
        self.model = model
    
    def trainW2V(self, numWalks, walkLength, embedLength):
        sentences = test.corpus.simulate_walks(numWalks,walkLength)
        if (self.embedder == None):
            self.embedder = Word2Vec(sentences, size=embedLength, workers=4)
        else:
            self.embedder.build_vocab(more_sentences, update=True)
            self.embedder.train(sentences, total_examples=self.embedder.corpus_count, epochs=self.embedder.iter)
    
    def saveW2V(self, fname):
        self.embedder.save(fname)
    
    def loadW2V(self, fname):
        self.embedder = gensim.models.Word2Vec.load(fname)
    
    def trainModel(self):
        trainData = corpus.train
        pass
    
    def saveModel(self, fname):
        pass
    
    def loadModel(self, fname):
        pass
    
    def evaluate(self, setType = 'train'):
        pass
    
    

In [49]:
#note: place model into a seperate class with calls on Corpus

In [50]:
pd.read_csv("../data/gen/abt_buy_graph.csv")

,source,target,type
0,38477,10011646,ground_truth
1,38475,10140760,ground_truth
2,33053,10221960,ground_truth
3,27248,10246269,ground_truth
4,25262,10315184,ground_truth
...,...,...,...
62761,31992,NaN,NaN
62762,34697,NaN,NaN
62763,210252385,NaN,NaN
62764,38346,NaN,NaN


In [51]:
gensim.models.doc2vec.FAST_VERSION > -1


True

In [52]:
test = Corpus("../data/gen/abt_buy_graph.csv", '../data/Abt-Buy/abt_buy_perfectMapping.csv', 0.3, 1, 2)

building train/test sets
['idAbt', 'idBuy']
building graph


In [118]:
t = test.corpus.simulate_walks(1000,10)

Walk iteration:
1 / 1000
2 / 1000
3 / 1000
4 / 1000
5 / 1000
6 / 1000
7 / 1000
8 / 1000
9 / 1000
10 / 1000
11 / 1000
12 / 1000
13 / 1000
14 / 1000
15 / 1000
16 / 1000
17 / 1000
18 / 1000
19 / 1000
20 / 1000
21 / 1000
22 / 1000
23 / 1000
24 / 1000
25 / 1000
26 / 1000
27 / 1000
28 / 1000
29 / 1000
30 / 1000
31 / 1000
32 / 1000
33 / 1000
34 / 1000
35 / 1000
36 / 1000
37 / 1000
38 / 1000
39 / 1000
40 / 1000
41 / 1000
42 / 1000
43 / 1000
44 / 1000
45 / 1000
46 / 1000
47 / 1000
48 / 1000
49 / 1000
50 / 1000
51 / 1000
52 / 1000
53 / 1000
54 / 1000
55 / 1000
56 / 1000
57 / 1000
58 / 1000
59 / 1000
60 / 1000
61 / 1000
62 / 1000
63 / 1000
64 / 1000
65 / 1000
66 / 1000
67 / 1000
68 / 1000
69 / 1000
70 / 1000
71 / 1000
72 / 1000
73 / 1000
74 / 1000
75 / 1000
76 / 1000
77 / 1000
78 / 1000
79 / 1000
80 / 1000


KeyboardInterrupt: 

In [ ]:
mdl = gensim.models.Word2Vec(sentences = t, workers=-1, size=100, sg = 1, hs = 0, negative = 10)

In [ ]:
df = test.train
embedding1 = mdl[[str(x) for x in df.idAbt.values]]
embedding2 = mdl[[str(x) for x in df.idBuy.values]]
data = np.concatenate([embedding1,embedding2], axis = 1)

In [112]:
df2 = test.test
embedding1 = mdl[[str(x) for x in df2.idAbt.values]]
embedding2 = mdl[[str(x) for x in df2.idBuy.values]]
data2 = np.concatenate([embedding1,embedding2], axis = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [90]:
from sklearn.svm import SVC

In [113]:
model = SVC()

In [115]:
model.fit(data.reshape(-1, 1), df.label)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [116]:
model.score(data.reshape(-1, 1), df.label)

0.533204384268214

In [117]:
model.score(data2.reshape(-1, 1), df2.label)

0.48522550544323484

In [ ]:
204238996 in pd.read_csv('../data/Abt-Buy/abt_buy_perfectMapping.csv').idBuy

In [ ]:
pd.read_csv('../data/Abt-Buy/abt_buy_perfectMapping.csv')

In [ ]:
test.corpus.simulate_walks(10, 10)

In [ ]:
test.test.label.mean()

In [ ]:
test.train.label.mean()

In [ ]:
df = pd.read_csv("../data/gen/abt_buy_graph.csv")

In [ ]:
temp1 = df[df['type'] == 'ground_truth']
temp2 = df[df['type'] != 'ground_truth']

In [ ]:
temp1 = temp1.drop(columns = 'type')

In [ ]:
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

In [ ]:
graph[0][4]

In [ ]:
Graph()

In [ ]:
allItems = set(temp2.source)

In [ ]:
G=nx.Graph()

In [ ]:
temp = set(df.source)
len(temp)

In [ ]:
temp.update(set(df.target))

In [ ]:
temp = [str(x) for x in list(temp)]

In [ ]:
len(temp)

In [ ]:
G.add_nodes_from(temp)

In [ ]:
edges = tuple(zip([str(x) for x in list(df['source'])], [str(x) for x in list(df['target'])]))

In [ ]:
G.add_edges_from(edges)

In [ ]:
list(G.neighbors('nan'))

In [ ]:
#G.nodes

In [ ]:
#temp

In [ ]:
df = pd.read_csv("../data/gen/abt_buy_graph.csv")

In [ ]:
df[['target','type']]

In [ ]:
utils.simple_preprocess('cat dog')

In [ ]:

sentences = ['dog, cat']
model = gensim.models.Word2Vec(sentences=utils.simple_preprocess('cat dog'))